In [1]:
# 匯入warnings模組，用於忽略不必要的警告訊息
import warnings

# 忽略所有的警告訊息，讓輸出結果更乾淨
warnings.filterwarnings("ignore")

# 引入pkg_resources模組，用於取得目前環境中已安裝的套件資訊
import pkg_resources

# 設定一個要查詢的套件清單，用於比對目前已安裝的套件
packages_to_check = ['keras', 'tensorflow', 'gym']

# 逐一檢查目前環境中所有已安裝的套件
for dist in pkg_resources.working_set:
    # 取得套件名稱並轉成小寫，以便進行不區分大小寫的比對
    name = dist.project_name.lower()

    # 判斷目前的套件名稱是否包含在我們要查詢的清單中
    if any(pkg in name for pkg in packages_to_check):
        # 如果符合條件，則印出該套件的名稱和版本號
        print(f"{dist.project_name} == {dist.version}")

gym == 0.26.2
gym-notices == 0.1.0
gymnasium == 1.1.1
keras == 3.10.0
tensorflow == 2.20.0


In [2]:
# 匯入numpy套件，用於數值計算與陣列處理
import numpy as np

# 解決numpy版本更新後移除np.bool和np.bool8的問題
if not hasattr(np, 'bool'):   # 如果np模組中沒有bool屬性，則將其指向內建的bool類型
    np.bool = bool
if not hasattr(np, 'bool8'):  # 如果np模組中沒有bool8屬性，則將其指向np.bool
    np.bool8 = np.bool

# 匯入os模組，用於與作業系統互動
import os

# 指定Keras使用TensorFlow作為後端運算引擎
os.environ["KERAS_BACKEND"] = "tensorflow"

# 匯入gym套件，用於建立和操作強化學習環境
import gym

# 匯入TensorFlow套件，用於深度學習模型的建構與訓練
import tensorflow as tf

# 從TensorFlow中匯入Keras模組，用於建構和訓練神經網路模型
from tensorflow import keras

# 從Keras中匯入layers模組，用於建立神經網路的各種層(例如：Dense、Conv等)
from tensorflow.keras import layers

# 從Keras中匯入backend模組，提供底層張量操作功能(例如：log、expand_dims等)
from tensorflow.keras import backend as ops

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
# 設定強化學習的超參數
seed = 42                      # 隨機種子，確保實驗可重現
gamma = 0.99                   # 折扣因子，影響未來獎勵的權重
max_steps_per_episode = 10000  # 每個回合的最大步數，防止無限迴圈

# 建立CartPole-v0環境
env = gym.make("CartPole-v0")

# 重設環境並設定隨機種子
env.reset(seed = seed)

# 取得float32類型的最小正數，用於避免除以零或數值不穩定的錯誤
eps = np.finfo(np.float32).eps.item()

# 定義神經網路的結構與參數
num_inputs = 4    # CartPole環境的觀測空間維度
num_actions = 2   # CartPole環境的動作空間維度(左或右)
num_hidden = 128  # 神經網路隱藏層的單元數

# 建立輸入層，輸入形狀為(num_inputs,)，代表神經網路接受的狀態空間維度
inputs = layers.Input(shape = (num_inputs,))

# 建立一個全連接層，包含num_hidden個神經元，激活函數為ReLU
common = layers.Dense(num_hidden, activation="relu")(inputs)

# 建立行動輸出層(Actor)，包含num_actions個神經元，使用softmax激活函數輸出行動機率分布
action = layers.Dense(num_actions, activation="softmax")(common)

# 建立評估輸出層(Critic)，包含1個神經元，輸出狀態價值估計
critic = layers.Dense(1)(common)

# 建立整體模型，輸入為inputs，輸出包含action和critic兩個分支
model = keras.Model(inputs = inputs, outputs = [action, critic])

# 定義Adam優化器，學習率設為0.01
optimizer = keras.optimizers.Adam(learning_rate = 0.01)

# 定義Huber損失函數，用於計算Critic的損失
huber_loss = keras.losses.Huber()

# 用於記錄每個時間步的行動機率、Critic預測值和獎勵
action_probs_history = []
critic_value_history = []
rewards_history = []

# 初始化累積獎勵和回合計數器，方便追蹤訓練進度
running_reward = 0
episode_count = 0

# 持續訓練直到任務被解決(例如達到指定分數)
while True:
    # 重置環境並取得初始狀態(reset傳回值為tuple，新版gym需取[0])
    state = env.reset()[0]

    # 初始化本回合的累積獎勵
    episode_reward = 0

    # 使用TensorFlow的GradientTape追蹤梯度，方便後續反向傳播
    with tf.GradientTape() as tape:
        # 在每個回合中持續與環境互動，直到達到最大步數或遊戲結束
        for timestep in range(1, max_steps_per_episode):
            # 將環境狀態轉成張量，並新增batch維度以符合模型輸入格式
            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)

            # 呼叫模型，輸出行動機率(action_probs)與狀態價值估計(critic_value)
            action_probs, critic_value = model(state)

            # 記錄評估器預測的狀態價值
            critic_value_history.append(critic_value[0, 0])

            # 根據行動機率分布，隨機採取一個動作
            action = np.random.choice(num_actions, p = np.squeeze(action_probs.numpy()))

            # 記錄所採樣行動的log機率，用於計算策略梯度
            action_probs_history.append(ops.log(action_probs[0, action]))

            # 根據選定的動作與當前狀態與環境互動，取得新狀態與獎勵
            state, reward, done, *_ = env.step(action)

            # 紀錄獲得的獎勵
            rewards_history.append(reward)

            # 累積本回合的獎勵
            episode_reward += reward

            # 若回合結束，則跳出迴圈
            if done:
                break

        # 使用指數移動平均更新running_reward，作為學習進度指標
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # 用來存放每個時間步的折扣回報(未來回報的加權和)
        returns = []

        # 初始化折扣回報的累積和為0
        discounted_sum = 0

        # 反向遍歷歷史獎勵列表，計算每個時間點的折扣回報
        for r in rewards_history[::-1]:
            # 折扣回報 = 當前獎勵 + 折扣因子 * 之前的折扣回報
            discounted_sum = r + gamma * discounted_sum

            # 將計算結果插入列表開頭，保持時間順序
            returns.insert(0, discounted_sum)

        # 將折扣回報轉成numpy陣列
        returns = np.array(returns)

        # 對折扣回報進行標準化，減少數值偏差，提高訓練的穩定性
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)

        # 將標準化後的折扣回報轉換為列表格式，方便後續操作
        returns = returns.tolist()

        # 將行動機率歷史、Critic預測值和正規化折扣回報打包，方便後續計算損失
        history = zip(action_probs_history, critic_value_history, returns)

        # 初始化演員(Actor)損失和評論者(Critic)損失的列表
        actor_losses = []
        critic_losses = []

        # 遍歷每個時間步的資料，計算Actor與Critic的損失
        for log_prob, value, ret in history:
            # 計算差異：實際折扣回報與Critic預測值的差距
            diff = ret - value

            # Actor損失：使模型更偏向選擇帶來更高回報的行動，故乘以負對數機率與差異
            actor_losses.append(-log_prob * diff)

            # Critic損失：使用Huber損失函數衡量預測值與實際回報的誤差，幫助Critic學習更準確的價值估計
            critic_losses.append(huber_loss(ops.expand_dims(value, 0), ops.expand_dims(ret, 0)))

        # 計算總損失：Actor損失加上Critic損失
        loss_value = sum(actor_losses) + sum(critic_losses)

        # 計算損失對模型參數的梯度
        grads = tape.gradient(loss_value, model.trainable_variables)

        # 使用優化器根據梯度更新模型參數
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # 清空儲存行動機率、Critic值與獎勵的歷史資料，為下一回合做準備
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # 紀錄已完成的回合數
    episode_count += 1

    # 每執行10回合，輸出目前的平均獎勵(running_reward)與當前回合數，方便觀察訓練進展
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))

    # 若移動平均獎勵超過195，代表模型已成功完成任務，輸出回合數並結束訓練
    if running_reward > 195:
        print("Solved at episode {}!".format(episode_count))
        break

running reward: 11.57 at episode 10
running reward: 32.05 at episode 20
running reward: 46.81 at episode 30
running reward: 69.23 at episode 40
running reward: 133.63 at episode 50
running reward: 157.21 at episode 60
Solved at episode 62!
